In [1]:
(ns vlado)

nil


In [2]:
(def program "1 2 + 4 +")

#'vlado/program


In [3]:
(read-string (str "[" program "]"))

[1 2 + 4 +]


In [4]:
(defprotocol IEvaluate
    "protocol for evaluation of prgram"
    (-eval [this state]))

IEvaluate


In [5]:
(defrecord State [ds ps env])

vlado.State


In [6]:
(extend-protocol IEvaluate
    java.lang.Long
    (-eval [this state]
        (update-in state [:ds] conj this))
    clojure.lang.Symbol
    (-eval [this state]
        (let [[a b & ds] (:ds state)
              f (this (:env state))]
          (f state)
)))
;;(extends? java.lang.Long IEvaluate)

nil


In [7]:
(defn wrap-function [f num-args]
    (fn [{:keys [ds ps] :as state}]
        (let [args (take num-args ds)
              rest-args (drop num-args ds)]
              (assoc state :ds (vec (concat [(apply f args)] rest-args))))))

#'vlado/wrap-function


In [8]:
(defn init-keywords []
    {'+ (wrap-function + 2)
     '- (wrap-function - 2)
     '* (wrap-function * 2)
    }
)

#'vlado/init-keywords


In [9]:
(defn init-state [program]
    (State. [] (read-string (str "[" program "]")) (init-keywords)))

;;(init-state program)

#'vlado/init-state


In [10]:
(defn interpret [program]
    (loop [state (init-state program)]
        (let [ch (first (:ps state))
              state (update-in state [:ps] rest)]
            (if ch
                (recur (-eval ch state))
                (first (:ds state))
))))

(interpret program)

#'vlado/interpret
7


In [13]:
(interpret "1 2 3 + *")

9
